In [8]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [9]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [10]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [11]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [12]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfail'], pff=False)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350, resolution=N

In [13]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False)
IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False)
IntegerParameter('EWS_DaysToThreat', 0, 4, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False)
IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False)
IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False)
IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIncrease 0'

In [14]:
from funs_policy_eval import create_optimization_policies, create_collaborative_policies

proposed_policies_files = ["OptimizationGel1_4Pol_v1.csv","OptimizationResults_DR12_v1.csv"]


#DR12_OP_v1 = create_optimization_policies("OptimizationResults_DR12_v1.csv","DR12_v1_P",dike_model)
#GEL_OP_v1 = create_optimization_policies("OptimizationGel1_4Pol_v1.csv","GEL_OP_v1",dike_model)
collaborative_policies = create_collaborative_policies("Policies and DR description for the viz - Policies.csv",dike_model)


policies=collaborative_policies

In [15]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 48 policies * 1 model(s) = 4800 experiments
100%|██████████████████████████████████████| 4800/4800 [03:45<00:00, 21.26it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [16]:
experiments, outcomes = results
print(experiments.keys())
experiments.head()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', '0_RfR 0', '1_RfR 0',
       '2_RfR 0', '3_RfR 0', '4_RfR 0', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.2_DikeIncrease 0', 'A.3_DikeIncrease 0',
       'A.4_DikeIncrease 0', 'A.5_DikeIncrease 0', 'scenario', 'policy',
       'model'],
      dtype='object')


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,scenario,policy,model
0,66,256.405873,10.0,0.914050,62.137988,1.0,0.745812,196.855064,10.0,0.714089,...,0,0,5,0,0,0,0,0,Policy 1,dikesnet
1,91,160.630517,1.0,0.637861,67.739713,10.0,0.513400,108.299653,1.0,0.643795,...,0,0,5,0,0,0,0,1,Policy 1,dikesnet
2,110,185.886840,1.0,0.908846,183.624915,1.5,0.795167,190.933506,1.0,0.696647,...,0,0,5,0,0,0,0,2,Policy 1,dikesnet
3,105,265.181037,1.0,0.773226,119.844979,1.0,0.109701,283.506646,1.0,0.138487,...,0,0,5,0,0,0,0,3,Policy 1,dikesnet
4,77,70.041965,1.0,0.877746,99.057114,1.5,0.399784,217.204955,1.5,0.356421,...,0,0,5,0,0,0,0,4,Policy 1,dikesnet


In [17]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'EWS_DaysToThreat',
       'scenario', 'policy', 'model', 'RfR Total Costs',
       'Expected Evacuation Costs', 'Dike ring', 'Bmax', 'Brate', 'pfail',
       'RfR 0', 'DikeIncrease 0', 'Dike Investment Costs',
       'Expected Annual Damage', 'Expected Number of Deaths'],
      dtype='object')


,A.0_ID flood wave shape,discount rate 0,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,Dike ring,Bmax,Brate,pfail,RfR 0,DikeIncrease 0,Dike Investment Costs,Expected Annual Damage,Expected Number of Deaths
0,66,2.5,0,0,Policy 1,dikesnet,84600000.0,0.0,A.1,256.405873,10.0,0.914050,1,5,5.397251e+07,0.0,0.0
1,91,1.5,0,1,Policy 1,dikesnet,84600000.0,0.0,A.1,160.630517,1.0,0.637861,1,5,5.397251e+07,0.0,0.0
2,110,1.5,0,2,Policy 1,dikesnet,84600000.0,0.0,A.1,185.886840,1.0,0.908846,1,5,5.397251e+07,0.0,0.0
3,105,1.5,0,3,Policy 1,dikesnet,84600000.0,0.0,A.1,265.181037,1.0,0.773226,1,5,5.397251e+07,0.0,0.0
4,77,1.5,0,4,Policy 1,dikesnet,84600000.0,0.0,A.1,70.041965,1.0,0.877746,1,5,5.397251e+07,0.0,0.0
